<a href="https://colab.research.google.com/github/Chutki319239/msazure/blob/main/Generative_Adversarial_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SIMPLE GAN**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

class Discriminator(nn.Module):
  def __init__(self,img_dim):
    super().__init__()
    self.disc=nn.Sequential(
        nn.Linear(img_dim, 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128, 1),
        nn.Sigmoid(),
    )

  def forward(self,x):
    return self.disc(x)

class Generator(nn.Module):
  def __init__(self,z_dim,img_dim):
    super().__init__()
    self.gen=nn.Sequential(
        nn.Linear(z_dim, 256),
        nn.LeakyReLU(0.1),
        nn.Linear(256, img_dim),
        nn.Tanh(),
    )

  def forward(self,x):
    return self.gen(x)

#Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28*28*1
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim , image_dim).to(device)
fixed_noise = torch.randn((batch_size,z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))]
    )
dataset = datasets.MNIST(root="dataset/",transform=transforms,download=True)
loader = DataLoader(dataset,batch_size=batch_size,shuffle=True)
opt_disc = optim.Adam(disc.parameters(),lr=lr)
opt_gen = optim.Adam(gen.parameters(),lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step=0

for epoch in range(num_epochs):
  for batch_idx,(real,_) in enumerate(loader):
    real = real.view(-1,784).to(device)
    batch_size = real.shape[0]

    ### Train Discriminator: max log(D(real)) + log(1-D(G(z)))
    noise = torch.randn(batch_size , z_dim).to(device)
    fake = gen(noise)
    disc_real = disc(real).view(-1)
    lossD_real = criterion(disc_real , torch.ones_like(disc_real))
    disc_fake = disc(fake).view(-1)
    lossD_fake = criterion(disc_fake , torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake)/2
    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()

    ### Train Generator: min log(1-D(G(z))) <--> max log(D(G(z)))
    output = disc(fake).view(-1)
    lossG = criterion(output , torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    if batch_idx == 0:
      print(
          f"Epoch [{epoch}/{num_epochs}] "
          f"Loss D: {lossD:.4f},Loss G: {lossG:.4f}"
      )

      with torch.no_grad():
        fake = gen(fixed_noise).reshape(-1,1,28,28)
        data = real.reshape(-1,1,28,28)
        img_grid_fake = torchvision.utils.make_grid(fake,normalize=True)
        img_grid_real = torchvision.utils.make_grid(data,normalize=True)

        writer_fake.add_image(
            "Mnist Fake Images",img_grid_fake,global_step=step
        )
        writer_real.add_image(
            "Mnist Real Images",img_grid_real,global_step=step
        )
        step += 1

















Epoch [0/50] Loss D: 0.6260,Loss G: 0.7125
Epoch [1/50] Loss D: 0.7449,Loss G: 0.6970
Epoch [2/50] Loss D: 0.7719,Loss G: 0.6223
Epoch [3/50] Loss D: 0.6116,Loss G: 1.0847
Epoch [4/50] Loss D: 0.4664,Loss G: 1.1113
Epoch [5/50] Loss D: 0.6287,Loss G: 0.8067
Epoch [6/50] Loss D: 0.9400,Loss G: 0.6169
Epoch [7/50] Loss D: 0.5510,Loss G: 0.8846
Epoch [8/50] Loss D: 0.6541,Loss G: 1.2470
Epoch [9/50] Loss D: 0.5093,Loss G: 1.4328
Epoch [10/50] Loss D: 0.4958,Loss G: 1.2896
Epoch [11/50] Loss D: 0.5488,Loss G: 0.9784
Epoch [12/50] Loss D: 0.7119,Loss G: 0.8661
Epoch [13/50] Loss D: 0.6342,Loss G: 0.9786
Epoch [14/50] Loss D: 0.4984,Loss G: 1.3031
Epoch [15/50] Loss D: 0.5594,Loss G: 1.1614
Epoch [16/50] Loss D: 0.8919,Loss G: 0.8889
Epoch [17/50] Loss D: 0.6274,Loss G: 1.1065
Epoch [18/50] Loss D: 0.8165,Loss G: 0.8084
Epoch [19/50] Loss D: 0.6106,Loss G: 1.0341
Epoch [20/50] Loss D: 0.5962,Loss G: 1.2318
Epoch [21/50] Loss D: 0.6083,Loss G: 1.0294
Epoch [22/50] Loss D: 0.4642,Loss G: 1.123

# **DCGAN IMPLEMENTATION FROM SCRATCH**

In [ ]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
  def __init__(self,channels_img,features_d):
    super(Discriminator,self).__init__()
    self.disc = nn.Sequential(
        #Input: N*channels_img*64*64
        nn.Conv2d(
            channels_img,features_d,kernel_size=4,stride=2,padding=1
        ), # 32*32
        nn.LeakyReLU(0.2),
        self._block(features_d,features_d*2,4,2,1), #16*16
        self._block(features_d*2,features_d*4,4,2,1), #8*8
        self._block(features_d*4,features_d*8,4,2,1), #4*4
        nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=0), #1*1
    )

  def _block(self,in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  def forward(self,x):
    return self.disc(x)



class Generator(nn.Module):
  def __init__(self,z_dim,channels_img,features_g):
    super(Generator,self).__init__()
    self.gen=nn.Sequential(
        #Input: N*z_dim*1*1
        self._block(z_dim,features_g*16,4,1,0), # N*f_g*16*4*4
        self._block(features_g*16,features_g*8,4,2,1), # 8*8
        self._block(features_g*8,features_g*4,4,2,1), # 16*16
        self._block(features_g*4,features_g*2,4,2,1), # 32*32
        nn.ConvTranspose2d(
            features_g*2,channels_img,kernel_size=4,stride=2,padding=1,
        ),
        nn.Tanh(), #[-1,1]

    )


  def _block(self,in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
    )
def forward(self,x):
  return self.gen(x)


def initialize_weights(model):
  for m in model.modules():
    if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data,0.0,0.02)

def test():
  N,in_channels,H,W=8,3,64,64
  z_dim=100
  x=torch.randn((N,in_channels,H,W))
  disc=Discriminator(in_channels,8)
  initialize_weights(disc)
  assert disc(x).shape == (N,1,1,1)
  gen=Generator(z_dim,in_channels,8)
  initialize_weights(gen)
  z=torch.randn(N,z_dim,1,1)
  print("Success")

test()





Success


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

class Discriminator(nn.Module):
  def __init__(self,channels_img,features_d):
    super(Discriminator,self).__init__()
    self.disc = nn.Sequential(
        #Input: N*channels_img*64*64
        nn.Conv2d(
            channels_img,features_d,kernel_size=4,stride=2,padding=1
        ), # 32*32
        nn.LeakyReLU(0.2),
        self._block(features_d,features_d*2,4,2,1), #16*16
        self._block(features_d*2,features_d*4,4,2,1), #8*8
        self._block(features_d*4,features_d*8,4,2,1), #4*4
        nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=0), #1*1
        nn.Sigmoid(),
    )

  def _block(self,in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  def forward(self,x):
    return self.disc(x)

class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N*z_dim*1*1
            self._block(z_dim, features_g * 16, 4, 1, 0),  # N*f_g*16*4*4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # 8*8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # 16*16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # 32*32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1,
            ),
            nn.Tanh(),  # [-1,1]
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    # The forward function needs to be inside the Generator Class
    def forward(self, x):
        return self.gen(x)

#Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)],[0.5 for _ in range(CHANNELS_IMG)]),
    ]
)

dataset = datasets.MNIST(root="dataset/",train=True,transform=transforms,download=True)
loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True)
gen = Generator(Z_DIM,CHANNELS_IMG,FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG,FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(),lr=LEARNING_RATE,betas=(0.5,0.999))
opt_disc = optim.Adam(disc.parameters(),lr=LEARNING_RATE,betas=(0.5,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32,Z_DIM,1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step=0

gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
  for batch_idx,(real,_) in enumerate(loader):
    real=real.to(device)
    noise=torch.randn((BATCH_SIZE,Z_DIM,1,1)).to(device)
    fake=gen(noise)

    ###Train Discriminator max log(D(x)) + log (1-D(G(z)))
    disc_real=disc(real).reshape(-1)
    loss_disc_real=criterion(disc_real,torch.ones_like(disc_real))
    disc_fake=disc(fake.detach()).reshape(-1)
    loss_disc_fake=criterion(disc_fake,torch.zeros_like(disc_fake))
    loss_disc=(loss_disc_real+loss_disc_fake)/2
    disc.zero_grad()
    loss_disc.backward(retain_graph=True)
    opt_disc.step()

    ###Train Generator min log(1-D(G(z))) <--> max log(D(G(z)))
    output=disc(fake).reshape(-1)
    loss_gen=criterion(output,torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()

    #Print losses occasionally and print to tensorboard
    if batch_idx % 100 == 0:
      print(
          f"Epoch[{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \
          Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
          )
      with torch.no_grad():
        fake=gen(fixed_noise)
        #take out (up to) 32 examples
        img_grid_real = torchvision.utils.make_grid(real[:32],normalize=True)
        img_grid_fake = torchvision.utils.make_grid(fake[:32],normalize=True)
        writer_real.add_image("Real",img_grid_real,global_step=step)
        writer_fake.add_image("Fake",img_grid_fake,global_step=step)

      step += 1






Epoch[0/5] Batch 0/469           Loss D: 0.7008, loss G: 0.7720
Epoch[0/5] Batch 100/469           Loss D: 0.0150, loss G: 4.1149
